In [ ]:
import requests, json
import pandas as pd
import numpy as np

# Get category ids from Greg's spreadsheet

In [ ]:
df = pd.read_csv('gregr_report.csv')
df.head()

,Catalog Id,Bib Marc Tag Number,Bib Marc Subfield Name,Bib Marc Subfield Data
0,2,500,a,Includes index.
1,2,504,a,"""Works of C. G. Jung"": p. 201-204. Bibliograph..."
2,591,500,a,"Description based on: Vol. 1, no. 4; title fro..."
3,591,500,a,"Published: New York : Plenum Press, 1979-"
4,976,500,a,Includes index.


In [ ]:
vals = set(df.to_numpy()[:, 0])
cat_val = []

for val in vals:
  cat_val.append(val)

category_ids = np.array(cat_val)
print(category_ids)

[      2 5242885 5505056 ... 4063193 4063194 4063195]


In [ ]:
# Indicies that need parsing: 3 subjects, 4 notes, 7
def parse_data_values(record):
  pass


def parse_subjects(record):
  full_str = ""
  for i, dic in enumerate(record):
    full_str += dic['value']

    if i != len(record) - 1:
      full_str += ", "

  return full_str


def parse_notes(record):
  
  full_str = ""
  for i, dic in enumerate(record):
    
    
    # Skip bibliographical data
    if "label" in dic:
      if dic['label'] == "Bibliography":
        continue

      # Join strings in a list if they exist
      list_str = ""

      list_str = ", ".join(dic['values'])

      full_str += dic['label'] + ": " + list_str + "\n"

      if i != len(record) - 1:
        full_str += ", "
    
  
  return full_str


# What should we do in terms of handling multiple editions/holdings (include ebook data?)
def parse_holdings(record):
  call_number = None
  for i, dic in enumerate(record):


    # Get call number for print copies only
    if dic['holdingType'] == "book":

      # Check if there is a call number key in teh dictionary
      if "callNumber" in dic:

        # Save call number and exit loop
        call_number = dic['callNumber']
        break
  
  # If no call number is found, save None as placeholder
  if call_number is None:
    call_number = "None"

  return call_number

def parse_publishers(record):
  publisher_str = ""

  for j, publisher_dict in enumerate(record):
    curr_pub = ""
    for i, key in enumerate(publisher_dict.keys()):
      curr_pub += publisher_dict[key]
      
      if i != len(publisher_dict) - 1:
        curr_pub += ", "
    
    publisher_str += curr_pub

    if j != len(record) - 1:
      publisher_str += ","

  return publisher_str

In [ ]:
'''
titleID - for reference
Maybe creators?
fullTitle
subjects
publicationDate = for reference
notes
thumbnail
holdings (for call number, maybe holding type later?)
'''

In [ ]:
all_data = []
fields_to_save = ["titleId", "fullTitle", "subjects", "publicationDate", "notes", "thumbnail", "holdings"]
url_base = "https://search.lib.byu.edu/green/byu/record/cat."

f = open('library_json.csv', 'w')
fields = '_'.join(fields_to_save)
fields += '\n'

f.write(fields)

for num, id in enumerate(category_ids):
  print("Entry number {} / {}".format(num, len(category_ids)))
  url = requests.get(url_base + str(id))
  text = url.text
  data = json.loads(text)

  data_vals = [None for i in range(7)]

  # Iterate over every field in json string from website
  for field in data:

    # Check for selected fields that I have decided to save
    if field == "titleId":
      data_vals[0] = data[field]
    elif field == "fullTitle":
      data_vals[1] = data[field]
    elif field == "subjects":
      data_vals[2] = data[field]
    elif field == "publicationDate":
      data_vals[3] = data[field]
    elif field == "notes":
      data_vals[4] = data[field]
    elif field == "thumbnail":
      data_vals[5] = data[field]
    elif field == "holdings":
      data_vals[6] = data[field]


  
  # Use functions to extract desired information from these json headers and format it
  if data_vals[2] is not None:
    data_vals[2] = parse_subjects(data_vals[2])
  if data_vals[4] is not None:
    data_vals[4] = parse_notes(data_vals[4])
  if data_vals[6] is not None:
    data_vals[6] = parse_holdings(data_vals[6])

  # Check for any blank fields and temporarily fill with None
  for i, value in enumerate(data_vals):
    if value == "" or value is None:
      data_vals[i] = "None"

  # Join data values with _ separator
  data = '\t'.join(data_vals)

  # Remove \n characters from data string
  data = data.replace('\n', '')

  # Add trailing newline
  data += '\n'

  # write to file
  f.write(data)

f.close()

Entry number 0 / 21413
Entry number 1 / 21413
Entry number 2 / 21413
Entry number 3 / 21413
Entry number 4 / 21413
Entry number 5 / 21413
Entry number 6 / 21413
Entry number 7 / 21413
Entry number 8 / 21413
Entry number 9 / 21413
Entry number 10 / 21413


KeyboardInterrupt: ignored

In [ ]:
all_data = []
fields_to_save = ["titleId", "fullTitle", "subjects", "publishers", "publicationDate", "notes", "thumbnail", "holdings"]
url_base = "https://search.lib.byu.edu/green/byu/record/cat."

f = open('library_json.csv', 'w')
fields = '\t'.join(fields_to_save)
fields += '\n'

f.write(fields)

for num, id in enumerate(category_ids):
  print("Entry number {} / {}".format(num, len(category_ids)))
  url = requests.get(url_base + str(id))
  text = url.text
  data = json.loads(text)

  data_vals = [None for i in range(8)]

  # Iterate over every field in json string from website
  for field in data:
    #print(field)

    # Check for selected fields that I have decided to save
    if field == "titleId":
      data_vals[0] = data[field]
    elif field == "fullTitle":
      data_vals[1] = data[field]
    elif field == "subjects":
      data_vals[2] = data[field]
    elif field == "publishers":
      data_vals[3] = data[field]
    elif field == "publicationDate":
      data_vals[4] = data[field]
    elif field == "notes":
      data_vals[5] = data[field]
    elif field == "thumbnail":
      data_vals[6] = data[field]
    elif field == "holdings":
      data_vals[7] = data[field]
  
  
  # Use functions to extract desired information from these json headers and format it
  if data_vals[2] is not None:
    data_vals[2] = parse_subjects(data_vals[2])
  if data_vals[3] is not None:
    data_vals[3] = parse_publishers(data_vals[3])
  if data_vals[5] is not None:
    data_vals[5] = parse_notes(data_vals[5])
  if data_vals[7] is not None:
    data_vals[7] = parse_holdings(data_vals[7])


  # Check for any blank fields and temporarily fill with None
  for i, value in enumerate(data_vals):
    if value == "" or value is None:
      data_vals[i] = "None"

  # Join data values with _ separator
  data = '\t'.join(data_vals)

  # Remove \n characters from data string
  data = data.replace('\n', '')

  # Add trailing newline
  data += '\n'

  # write to file
  f.write(data)

f.close()

Streaming output truncated to the last 5000 lines.
Entry number 16413 / 21413
Entry number 16414 / 21413
Entry number 16415 / 21413
Entry number 16416 / 21413
Entry number 16417 / 21413
Entry number 16418 / 21413
Entry number 16419 / 21413
Entry number 16420 / 21413
Entry number 16421 / 21413
Entry number 16422 / 21413
Entry number 16423 / 21413
Entry number 16424 / 21413
Entry number 16425 / 21413
Entry number 16426 / 21413
Entry number 16427 / 21413
Entry number 16428 / 21413
Entry number 16429 / 21413
Entry number 16430 / 21413
Entry number 16431 / 21413
Entry number 16432 / 21413
Entry number 16433 / 21413
Entry number 16434 / 21413
Entry number 16435 / 21413
Entry number 16436 / 21413
Entry number 16437 / 21413
Entry number 16438 / 21413
Entry number 16439 / 21413
Entry number 16440 / 21413
Entry number 16441 / 21413
Entry number 16442 / 21413
Entry number 16443 / 21413
Entry number 16444 / 21413
Entry number 16445 / 21413
Entry number 16446 / 21413
Entry number 16447 / 21413
Entr

In [ ]:
all_data = []
fields_to_save = ["titleId", "workTitle", "subjects", "publicationDate", "notes", "thumbnail", "holdings"]
url_base = "https://search.lib.byu.edu/green/byu/record/cat."

f = open('library_json.csv', 'w')
fields = '_'.join(fields_to_save)
fields += '\n'

f.write(fields)

for num, id in enumerate(category_ids):
  print("Entry number {} / {}".format(num, len(category_ids)))
  url = requests.get(url_base + str(id))
  text = url.text
  data = json.loads(text)

  data_vals = [None for i in range(7)]

  # Iterate over every field in json string from website
  for field in data:

    # Check for selected fields that I have decided to save
    if field == "titleId":
      data_vals[0] = data[field]
    elif field == "workTitle":
      data_vals[1] = data[field]
    elif field == "subjects":
      data_vals[2] = data[field]
    elif field == "publicationDate":
      data_vals[3] = data[field]
    elif field == "notes":
      data_vals[4] = data[field]
    elif field == "thumbnail":
      data_vals[5] = data[field]
    elif field == "holdings":
      data_vals[6] = data[field]

  
  # Use functions to extract desired information from these json headers and format it
  if data_vals[2] is not None:
    data_vals[2] = parse_subjects(data_vals[2])
  if data_vals[4] is not None:
    data_vals[4] = parse_notes(data_vals[4])
  if data_vals[6] is not None:
    data_vals[6] = parse_holdings(data_vals[6])

  # Check for any blank fields and temporarily fill with None
  for i, value in enumerate(data_vals):
    if value == "" or value is None:
      data_vals[i] = "None"

  # Join data values with _ separator
  data = '\t'.join(data_vals)

  # Remove \n characters from data string
  data = data.replace('\n', '')

  # Add trailing newline
  data += '\n'

  # write to file
  f.write(data)

f.close()

Streaming output truncated to the last 5000 lines.
Entry number 16413 / 21413
Entry number 16414 / 21413
Entry number 16415 / 21413
Entry number 16416 / 21413
Entry number 16417 / 21413
Entry number 16418 / 21413
Entry number 16419 / 21413
Entry number 16420 / 21413
Entry number 16421 / 21413
Entry number 16422 / 21413
Entry number 16423 / 21413
Entry number 16424 / 21413
Entry number 16425 / 21413
Entry number 16426 / 21413
Entry number 16427 / 21413
Entry number 16428 / 21413
Entry number 16429 / 21413
Entry number 16430 / 21413
Entry number 16431 / 21413
Entry number 16432 / 21413
Entry number 16433 / 21413
Entry number 16434 / 21413
Entry number 16435 / 21413
Entry number 16436 / 21413
Entry number 16437 / 21413
Entry number 16438 / 21413
Entry number 16439 / 21413
Entry number 16440 / 21413
Entry number 16441 / 21413
Entry number 16442 / 21413
Entry number 16443 / 21413
Entry number 16444 / 21413
Entry number 16445 / 21413
Entry number 16446 / 21413
Entry number 16447 / 21413
Entr

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Taking "gfg input file.txt" as input file
# in reading mode
with open("gfg input file.txt", "r") as input:
      
    # Creating "gfg output file.txt" as output
    # file in write mode
    with open("gfg output file.txt", "w") as output:
          
        # Writing each line from input file to
        # output file using loop
        for line in input:
            output.write(line)

In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')

In [ ]:
df = pd.read_csv("library_json_title_only.csv", sep="\t")
df.head()

FileNotFoundError: ignored

In [ ]:
df = df.drop(['titleId','publicationDate', 'thumbnail'], axis=1)
df.head()

,fullTitle,subjects,notes,holdings
0,"Complex, archetype, symbol in the psychology o...","Jung, C. G. (Carl Gustav), 1875-1961, Psychoan...",General Note: Includes index.,BF 174 .J85 J2936
1,Hexen- und Dämonenglaube im Lande Braunschweig...,"Witchcraft -- Germany -- Braunschweig Region, ...",None,BF 1583 .L44 1997
2,Qualitative strategies for ethnocultural resea...,"Psychology -- Qualitative research, Ethnology ...",Choice Review: This is an important book edite...,BF 76.5 .Q355 2012
3,"The art of persuading people, by James A. Wors...","Selling, Psychology, Applied","General Note: ""First edition.""",BF 636 .W6
4,Trauma and organizations / edited by Earl Hopper.,"Associations, institutions, etc, Stress (Psych...",Summary: This collection of new contributions ...,BF 175.5 .P75 T73x 2012


In [ ]:
df.to_csv('library_data_subjects_notes.csv')